In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
image_path = "/kaggle/input/apparel-images-dataset/"
folders = os.listdir(image_path)
print(folders)

In [ ]:
# Dataset X [color, type]
img_shp = 256

In [ ]:
cloths = {}
colors = {}
col_id = 0
cloth_id = 0
for folder in folders:
    color, cloth = folder.split('_')
    if color not in colors:
        colors[color] = col_id
        col_id += 1
    if cloth not in cloths:
        cloths[cloth] = cloth_id
        cloth_id += 1
        
print(colors, cloths)

In [ ]:
from skimage import io
from skimage.transform import resize

x = []
y = []
i = 0
for folder in folders:
    color, cloth = folder.split('_')
    cloth = cloths[cloth]
    color = colors[color]
    
    path =  image_path + folder + "/"
    for file in os.listdir(path):
        print(i)
        i+=1
        fpath = path + file
        x.append([color, cloth])
        
        # image load
        img = resize(io.imread(fpath), (64,64,3))
        y.append(img)
x = np.array(x)
y = np.array(y)

In [ ]:
rev_colors = {colors[k]:k for k in colors }
rev_cloths = {cloths[k]:k for k in cloths }
def get_reverse_map(data):
    return rev_colors[data[0]], rev_cloths[data[1]]

In [ ]:
import matplotlib.pyplot as plt

def plot_img(image):
    plt.axis("off")
    image = np.array(image).reshape((64,64,3))
    plt.imshow(image)
    plt.show()

idx =  104
x_sample = x[idx]
print(get_reverse_map(x_sample))
plot_img(y[idx])

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *

In [ ]:
def build_generator():
    n_nodes = 128*8*8
    
    # Image shape = 28,28,1
    z = Input(shape=(100,))      # latent vector = (100,1)
    z_i = Dense(n_nodes)(z)
    z_i = Reshape((8,8,128))(z_i)
    
    
    y = Input(shape=(6,))      # Color shape = (6,1)
    y_i = Dense(64)(y)  # 49,1
    y_i = Reshape((8,8,1))(y_i) 
    
    v = Input(shape=(5,))      # Cloth shape = (5,1)
    v_i = Dense(64)(v)  # 49,1
    v_i = Reshape((8,8,1))(v_i) 
    
    merge = Concatenate()([z_i, y_i, v_i]) # Output: (7,7,130)
    
    out = Conv2DTranspose(128, 4, 2, padding='same')(merge) # (14,14,128)
    out = LeakyReLU(alpha=0.2)(out)
    
    out = Conv2DTranspose(128, 4, 2, padding='same')(out) # (28,28,128)
    out = LeakyReLU(alpha=0.2)(out)
    
    out = Conv2DTranspose(256, 4, 2, padding='same')(out) # (56,56,256)
    out = LeakyReLU(alpha=0.2)(out)

    
    out = Conv2DTranspose(3, 7, 1, padding='same',activation='sigmoid')(out) # (64,64,3)
    
    model = Model([z,y, v],out)
    model.summary()
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy',optimizer=opt)
    return model
    
gen = build_generator()

In [ ]:
def build_discriminator(): 
    # Image shape = 28,28,1
    x = Input(shape=(64,64,3))  
    x_i = Conv2D(64,4,4,padding='same')(x)
    x_i = ReLU()(x_i)
    
    x_i = Conv2D(128,4,2,padding='same')(x_i)
    x_i = ReLU()(x_i)
    
    
    y = Input(shape=(6,))      # label shape = (10,1)
    y_i = Dense(64)(y)  # 49,1
    y_i = Reshape((8,8,1))(y_i) 
    
    v = Input(shape=(5,))      # Cloth shape = (5,1)
    v_i = Dense(64)(v)  # 49,1
    v_i = Reshape((8,8,1))(v_i) 
    
    merge = Concatenate()([x_i, y_i, v_i]) # Output: (7,7,129)
    out = Conv2D(256,4,2,padding='same')(merge)
    
    out = Flatten()(out)
    
    out = Dense(1024)(out) 
    
    out = Dense(512)(out) 
    out = ReLU()(out)
    
    out = Dense(256)(out) 
    out = ReLU()(out)
    
    out = Dense(1, activation='sigmoid')(out) 
    
    model = Model([x,y, v],out)
    model.summary()
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy',optimizer=opt)
    return model
    
disc = build_discriminator()   

In [ ]:
def build_gan(g,d):
    d.trainable = False
    # get generator inputs/outputs
    z, y, v = g.input
    x_g = g.output
    
    gan_output = d([x_g, y, v])
    model = Model([z, y, v], gan_output)
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    model.summary()
    return model
gan = build_gan(gen, disc)

In [ ]:
from keras.utils import to_categorical

In [ ]:
import random
def generate_real_samples(image, colors, cloths, n_samples):
    ix = np.random.randint(0, image.shape[0], n_samples)
    x, colors, cloths = image[ix], colors[ix], cloths[ix]
    y_ones = np.ones((n_samples,1))
    return [x, colors, cloths], y_ones

In [ ]:
def generate_latent_points(latent_dim, n_samples):
    x_input = np.random.randn(latent_dim*n_samples)
    z_input = x_input.reshape(n_samples, latent_dim)
    colors = to_categorical(np.random.randint(0, 6, n_samples),num_classes=6)
    cloths = to_categorical(np.random.randint(0, 5, n_samples),num_classes=5)
    return [z_input, colors, cloths]

In [ ]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    z_input, colors_input, cloths_input  = generate_latent_points(latent_dim, n_samples)
    images = generator.predict([z_input, colors_input, cloths_input])
    y = np.zeros((n_samples, 1))
    return [images, colors_input, cloths_input], y

In [ ]:
def train(g, d, gan, dataset, latent_dim, n_epochs=100, n_batch=128):
    bat_per_epo = int(dataset[0].shape[0]/n_batch)
    half_batch = int(n_batch / 2)
    for i in range(n_epochs):
        print("Epoch:", i)
        for j in range(bat_per_epo):
            # get random real samples
            [x_real, color_labels_real, cloth_labels_real],y_real = generate_real_samples(dataset[0], dataset[1], dataset[2], half_batch)
            # train discriminator
            d_loss1 = d.train_on_batch([x_real, color_labels_real, cloth_labels_real], y_real)
            
            # Generate fake samples
            [x_fake, color_labels, cloth_labels], y_fake = generate_fake_samples(g, latent_dim, half_batch)
            # train discriminator
            d_loss2 = d.train_on_batch([x_fake, color_labels, cloth_labels], y_fake)
            
            # prepare latent points 
            z_input, colors,cloths = generate_latent_points(latent_dim,n_batch)
            y_gan = np.ones((n_batch,1))
            
            # train gan
            g_loss = gan.train_on_batch([z_input, colors, cloths], y_gan)
            if j % 10 == 0:
                print("D1 Loss:",d_loss1, "D2 Loss:",d_loss2,"GAN Loss:",g_loss)
            
    g.save('cgan_generator.h5')
    d.save('cgan_discriminator.h5')
    gan.save('gan.h5')

In [ ]:
train_features = y
colors_labels, cloth_labels  = to_categorical(x[:,0], num_classes=6), to_categorical(x[:,1], num_classes=5)

In [ ]:
print(colors_labels.shape, cloth_labels.shape)

In [ ]:
train(gen, disc, gan, [train_features, colors_labels, cloth_labels], 100, n_epochs = 500)

In [ ]:
def generate_sample(g, n=30, latent=100):
    colors = to_categorical(np.array([x for x in range(0,6)]),num_classes=6)
    cloths = to_categorical(np.array([x for x in range(0,5)]),num_classes=5)
    z_vec = generate_latent_points(latent, 1)[0]
    columns = 10
    rows = n
    fig=plt.figure(figsize=(64, 64))
    k = 1
    for j in range(6):
        for i in range(5):
            z_vec = z_vec.reshape((1,100,1))
            color = colors[j].reshape((1,6,1))
            cloth = cloths[i].reshape((1,5,1))
            result = g.predict([z_vec,color,cloth])[0]
            ax = fig.add_subplot(rows, columns, k)
            ax.title.set_text(" ".join(get_reverse_map([j, i])))
            plt.imshow(result)
            k += 1
    plt.show()
    
generate_sample(gen)